In [1]:
BASE_ELASTIC_URL = "http://192.168.36.98/mytimes/elasticCommentQuery?sort=desc&appKey=TOI,ET&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED"
BATCH_SIZE_PARAM = 250
# ITER_BUFFER = 2
CSV_FILE_NAME = '/Users/yash.dalmia/elasticComments.csv'
DEFAULT_FIELDS = ['C_T']

In [2]:
startEpoch = None
endEpoch = None

In [3]:
import urllib.request, json 
import math
import csv
import pandas as pd

In [4]:
def getJsonFromUrl(url_param):
    with urllib.request.urlopen(url_param) as url:
        data = json.loads(url.read().decode())
        return data

In [5]:
def createNewCsv(file_path = CSV_FILE_NAME):
    with open(file_path, 'w+') as writeFile:
        pass

In [6]:
def getCsvListFromListComments(list_comments, fields_to_take):
    
    csv_list = []
    for x in list_comments:
        if 'C_T' in x:
            list1 = []
            for fieldStr in fields_to_take:
                if "." in fieldStr:
                    fieldStrArr = fieldStr.split(".")
                    if fieldStrArr[0] in x and fieldStrArr[1] in x[fieldStrArr[0]]:
                        list1.append(x[fieldStrArr[0]][fieldStrArr[1]])
                    else:
                        list1.append("")
                elif fieldStr in x:
                    list1.append(x[fieldStr])
                else:
                    list1.append("")
            csv_list.append(list1)
            
    return csv_list

In [7]:
def getModifiedUrl(url, from_param, size_param, startEpoch=None, endEpoch=None):
    result_url = url + "&from=" + str(from_param) + "&size=" + str(size_param)
    if (startEpoch!=None and endEpoch!=None):
        result_url = result_url + "&sDateEpoch=" + str(startEpoch) + "&eDateEpoch=" + str(endEpoch)
    return result_url

In [8]:
def getModifiedUrlDate(url, startEpoch=None, endEpoch=None):
#     result_url = url + "&from=" + str(from_param) + "&size=" + str(size_param)
    result_url = url
    if (startEpoch!=None and endEpoch!=None):
        result_url = result_url + "&sDateEpoch=" + str(startEpoch) + "&eDateEpoch=" + str(endEpoch)
    return result_url

In [9]:
def appendUrlResultToCsv(url, fields_to_take, size_param, file_path, force = False, to_download_till = None):
    
    final_url = getModifiedUrl(url, 0, size_param)
    json_raw_response = getJsonFromUrl(final_url)

    total_comment_count = json_raw_response['hits']['total']
    
    ELASTIC_LIMIT = 10000
    if (to_download_till != None and to_download_till < total_comment_count):
        total_comment_count = to_download_till
        
    if (total_comment_count > ELASTIC_LIMIT):
        print("total_comment_count greater than 10000: %d" %(total_comment_count))
        if ~force:
            print("aborting")
            return
        else:
            total_comment_count = ELASTIC_LIMIT
        
    
    num_iters = math.ceil(total_comment_count/size_param)
#     num_iters = num_iters + ITER_BUFFER

    print("total_count : %d" %(total_comment_count))
    print("num_iters : %d" %(num_iters))

    for iter_val in range(0, num_iters, 1):
        print("iteration : %d" %(iter_val))
        from_val = iter_val * size_param
        final_url = getModifiedUrl(url, from_val, size_param)
        json_raw_response = getJsonFromUrl(final_url)
        list_comments = [x['_source'] for x in json_raw_response['hits']['hits']]
        
        csv_list = getCsvListFromListComments(list_comments, fields_to_take)
        df_csv = pd.DataFrame(csv_list, columns = fields_to_take)
        
        with open(file_path, 'a', encoding='utf-8') as f:
            df_csv.to_csv(f, index = False, header=f.tell()==0, encoding='utf-8')

In [10]:
def generator_url_result(url, fields_to_take, size_param, file_path = None, force = False, to_download_till = None):
    
    final_url = getModifiedUrl(url, 0, size_param)
    json_raw_response = getJsonFromUrl(final_url)

    total_comment_count = json_raw_response['hits']['total']
    
    ELASTIC_LIMIT = 10000
    if (to_download_till != None and to_download_till < total_comment_count):
        total_comment_count = to_download_till
        
    if (total_comment_count > ELASTIC_LIMIT):
        print("total_comment_count greater than 10000: %d" %(total_comment_count))
        if ~force:
            print("aborting")
            return
        else:
            total_comment_count = ELASTIC_LIMIT
        
    
    num_iters = math.ceil(total_comment_count/size_param)
#     num_iters = num_iters + ITER_BUFFER

    print("total_count : %d" %(total_comment_count))
    print("num_iters : %d" %(num_iters))

    for iter_val in range(0, num_iters, 1):
        print("iteration : %d" %(iter_val))
        from_val = iter_val * size_param
        final_url = getModifiedUrl(url, from_val, size_param)
        json_raw_response = getJsonFromUrl(final_url)
        list_comments = [x['_source'] for x in json_raw_response['hits']['hits']]
        
        csv_list = getCsvListFromListComments(list_comments, fields_to_take)
        yield csv_list

In [11]:
# import re

In [12]:
# def clean_str_pure_hindi(string):
#     """
#     Tokenization/string cleaning for all datasets except for SST.
#     Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
#     """
#     string = re.sub(r"[^\u0900\u0901\u0902\u0903\u0904\u0905\u0906\u0907\u0908\u0909\u090a\u090b\u090c\u090d\u090e\u090f\u0910\u0911\u0912\u0913\u0914\u0915\u0916\u0917\u0918\u0919\u091a\u091b\u091c\u091d\u091e\u091f\u0920\u0921\u0922\u0923\u0924\u0925\u0926\u0927\u0928\u0929\u092a\u092b\u092c\u092d\u092e\u092f\u0930\u0931\u0932\u0933\u0934\u0935\u0936\u0937\u0938\u0939\u093a\u093b\u093c\u093d\u093e\u093f\u0940\u0941\u0942\u0943\u0944\u0945\u0946\u0947\u0948\u0949\u094a\u094b\u094c\u094d\u094e\u094f\u0950\u0951\u0952\u0953\u0954\u0955\u0956\u0957\u0958\u0959\u095a\u095b\u095c\u095d\u095e\u095f\u0960\u0961\u0962\u0963\u0964\u0965\u0966\u0967\u0968\u0969\u096a\u096b\u096c\u096d\u096e\u096f\u0970\u0971\u0972\u0973\u0974\u0975\u0976\u0977\u0978\u0979\u097a\u097b\u097c\u097d\u097e\u097f]", " ", string)

#     string = re.sub(r"\s{2,}", " ", string)
#     return string.strip()

In [13]:
import re

def filter_devanagiri(text):
    """
    filters for only devanagiri scripting language
    """
    #re.search returns None if not found
    mod_text = re.sub(r"[^\u0900-\u097F]", " ", text)
    mod_text = re.sub(r"\s{2,}", " ", mod_text)
    return mod_text.strip()

In [16]:
def run():
#     base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TOI&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&queryString=_exists_%3AC_T"
    base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=MTO&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&queryString=_exists_%3AC_T"
    
    csv_file_path = "/Users/yash.dalmia/MTO_cmts_30_march_20.csv"
    fields_to_take = ['c_id', 'C_T', 'C_APP_RES']

    batch_write_size = 250

    createNewCsv(file_path = csv_file_path)

    NET_END_TIME_EPOCH = 1585593000000
    NUM_DAYS_IN_ONE_ITER = 2
    NUM_ITERS = 2

    # endEpoch = NET_START_TIME_EPOCH
    startEpoch = NET_END_TIME_EPOCH

    for iter_val in range(0, NUM_ITERS, 1):
        print("\nIter_val : %d" %(iter_val))
        endEpoch = startEpoch
        startEpoch = endEpoch - int(NUM_DAYS_IN_ONE_ITER*24*60*60*1000)

        url = getModifiedUrlDate(base_url, startEpoch, endEpoch)
        print("url : " + url)
#         appendUrlResultToCsv(url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path)
        for csv_list in generator_url_result(url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path):
            print(csv_list[1])
            c_text = print(csv_list[1][1])
            print(c_text)
#             print(csv_list[1], old_regex(csv_list[1]), new_regex(csv_list[1]))

    print("\nnet startEpoch : ", startEpoch)
    print("net endEpoch : ", NET_END_TIME_EPOCH)
    print("csv written to %s"%(csv_file_path))

In [1]:
# run()

In [ ]:
# # url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=NBTO&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=24%2F4%2F2019&eDate=24%2F4%2F2019"
# url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDateEpoch=1548412448000&eDateEpoch=1556188448000&appKey=TOI&queryString=_exists_:SPAM_VAL%20AND%20SPAM_VAL.isSpam:false&multiActorId=6179270,665245713,14219596,6267354,665046059,549930791,548178454,668244979,663707220,22541448,2366863,2589915,655718924,526173678,659669241,103473849,2515704"
# from_param = 0
# size_param = 50
# # print(getModifiedUrl(url, from_param, size_param))
# csv_file_path = "data/most_abusive_TOI_last_90_days_25April.csv"
# fields_to_take = ['c_id', 'C_T', 'SPAM_VAL.round_prob']

# createNewCsv(file_path = csv_file_path)

# appendUrlResultToCsv(url, fields_to_take = fields_to_take, size_param = 250, file_path = csv_file_path)
# print('saved file at {}'.format(csv_file_path))

In [ ]:
# csv_file_path = "/Users/yash.dalmia/test_cmt.csv"
# createNewCsv(file_path = csv_file_path)

In [ ]:
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=MTO&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&actorInfo=vikasdc%40rediffmail.com&sDate=1%2F1%2F2019&queryString=_exists_%3AC_T"
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TIMESB&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=1%2F12%2F2019&eDate=31%2F1%2F2020&queryString=_exists_%3AC_T"
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TOI&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=23%2F2%2F2020&eDate=24%2F2%2F2020&queryString=_exists_%3AC_T"
# fields_to_take = ['C_T', 'C_D']
# fields_to_take = ['c_id','A_D_N', 'C_D', 'msid','A_K', 'C_T']
# batch_write_size = 250
# appendUrlResultToCsv(base_url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path)

In [ ]:
# df = pd.read_csv(csv_file_path)

In [ ]:
# df

In [ ]:
import time

In [ ]:
def convert_epoch_millis_to_date_string(epoch_millis):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(epoch_millis)/1000))

In [ ]:
# df['C_D'] = df['C_D'].apply(convert_epoch_millis_to_date_string)

In [ ]:
# df.drop(columns = ['F_NAME'], inplace=True)

In [ ]:
# df.to_csv(csv_file_path, encoding='utf-8', index=False)

In [ ]:
# import urllib

In [ ]:
# email_id_text = """
# meerameera9355@gmail.com
# chandukhan810@gmail.com
# bbbb07185@gmail.com
# asbn470@gmail.com
# meerarooy@gmail.com
# khanchandu751@gmail.com
# aasha3681@gmail.com
# mohammedkhan.mk395@gmail.com
# Ckha694@gmail.com
# """

In [ ]:
# email_id_list = [x for x in email_id_text.split('\n') if x]

In [ ]:
# email_id_list

In [ ]:
# fields_to_take = ['c_id', 'C_D', 'C_T', 'msid', 'IP', 'F_ADD', 'tet']
# batch_write_size = 250

In [ ]:
# for email in email_id_list:
#     print('email : {}'.format(email))
#     encoded_query_str= urllib.parse.quote(email)
#     csv_file_path = "/Users/yash.dalmia/Downloads/" + email + "_details.csv"
#     createNewCsv(file_path = csv_file_path)
#     base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TOI&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&actorInfo={}&queryString=_exists_%3AC_T".format(encoded_query_str)
#     appendUrlResultToCsv(base_url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path)

In [ ]:
# import time
# get_time = lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000))

In [ ]:
# df_r = pd.read_csv(csv_file_path)

In [ ]:
# df_r['time'] = df_r['C_D'].apply(get_time)

In [ ]:
# get_time(1560924551196)

In [ ]:
# df_r = df_r.drop(columns=['C_D'])

In [ ]:
# df_r.rename(index=str, columns={'C_T': 'Comment Text', 'time': 'Time'}, inplace=True)

In [ ]:
# df_r.to_csv(csv_file_path, index=False, encoding='utf-8')

In [ ]:
# df_r = pd.read_csv('/Users/yash.dalmia/toirb_all_cmts.csv')

In [ ]:
# df_r['C_T']